# Create OCEAN Data NFT
Quickly create OCEAN Data NFTs by querying blockchain data from Dune and Flipside.

**Steps:**
1. Get Data
2. Store Data
3. Create Data NFT

In [1]:
import json
import pandas as pd
import os
import time
from dotenv import load_dotenv
import requests
import subprocess
from flipside import Flipside
from ocean_lib.web3_internal.utils import connect_to_network
from ocean_lib.example_config import get_config_dict
from ocean_lib.ocean.ocean import Ocean
from brownie.network import accounts, chain

# Load variables from .env file
load_dotenv()
DUNE_API_KEY = os.getenv("DUNE_API_KEY")
FLIPSIDE_API_KEY = os.getenv("FLIPSIDE_API_KEY")

## 1. Get Data
### Flipside

* SDK Docs: https://docs.flipsidecrypto.com/flipside-api/get-started

In [2]:
# Query Flipside using their Python SDK
def query_flipside(sql):
    flipside = Flipside(FLIPSIDE_API_KEY, "https://api-v2.flipsidecrypto.xyz")
    results = flipside.query(sql)
    results_df = pd.DataFrame(results.rows, columns=results.columns).drop(
        columns=["__row_index"]
    )
    return results_df

In [3]:
sql = """
SELECT
  date(block_timestamp) as dt,
  count(distinct tx_hash) as tx_ct
FROM ethereum.core.fact_transactions
WHERE block_timestamp >= GETDATE() - interval'7 days'
GROUP BY 1
order by 1 asc
"""
results_df = query_flipside(sql)
results_df

,dt,tx_ct
0,2023-08-28T00:00:00.000Z,356396
1,2023-08-29T00:00:00.000Z,1019692
2,2023-08-30T00:00:00.000Z,1027316
3,2023-08-31T00:00:00.000Z,1010128
4,2023-09-01T00:00:00.000Z,1006635
5,2023-09-02T00:00:00.000Z,1024172
6,2023-09-03T00:00:00.000Z,1038128
7,2023-09-04T00:00:00.000Z,610216


### Dune

* API Docs: https://dune.com/docs/api/

In [6]:
# Query Dune Analytics via API
def query_dune(q):
    url = f"https://api.dune.com/api/v1/query/{q}/results?api_key={DUNE_API_KEY}"
    response = requests.get(url)
    results_json = json.loads(response.text)["result"]["rows"]
    results_df = pd.DataFrame.from_dict(results_json)
    return results_df

In [7]:
dune_query_id = 2847682  # Query URL: https://dune.com/queries/2847682
results_df = query_dune(dune_query_id)
results_df

KeyError: 'result'

## 2. Store Data
### Add File to GitHub

*Note: This is a quick & easy solution to host the data, but can also add file to IPFS, Arweave, GCP, AWS, etc...*

In [4]:
# Create File from Query results
results_df.to_csv("query_results.csv", index=False)

In [5]:
# Store file in GitHub repo
def git_add_commit_push(commit_message):
    try:
        subprocess.run(["git", "add", "."])
        subprocess.run(["git", "commit", "-m", commit_message])
        subprocess.run(["git", "push"])
        print("Git add, commit, and push successful!")
    except Exception as e:
        print("An error occurred:", e)


# Replace 'Your commit message here' with your desired commit message
commit_message = "Your commit message here"

git_add_commit_push(commit_message)

[main f4c50e5] Your commit message here
 2 files changed, 47 insertions(+), 33 deletions(-)
Git add, commit, and push successful!


To https://github.com/Demoniarc/ocean_data_nfts
   2593990..f4c50e5  main -> main


## 3. Create Data NFT
### Add Data NFT to the OCEAN Protocol Marketplace

* Marketplace: https://market.oceanprotocol.com/
* SDK Docs: https://docs.oceanprotocol.com/developers/ocean.py/install

In [6]:
# Connect to Blockchain
connect_to_network("polygon-main")
config = get_config_dict("polygon-main")
ocean = Ocean(config)

In [9]:
# Connect to Wallet
accounts.clear()
private_key = os.getenv("PRIVATE_KEY")
wallet = accounts.add(private_key)
print("MATIC Balance:", wallet.balance() / 1e18)

MATIC Balance: 7.769962592968272


In [10]:
# Create Data NFT
name = "Data NFT (Dune)"

# Link to File
result = subprocess.run(
    ["git", "config", "--get", "user.name"], stdout=subprocess.PIPE, text=True
)
github_username = result.stdout.strip().replace(" ", "")
url = f"https://raw.githubusercontent.com/{github_username}/ocean_data_nfts/main/query_results.csv"

# Set Gas Fees
priority_fee = chain.priority_fee
max_fee = (chain.base_fee * 2) + priority_fee
gas_limit = int((priority_fee + max_fee) / 1e5)
print("priority_fee:", priority_fee, "max_fee:", max_fee, "gas_limit:", gas_limit)

# Configure Transaction Variables
tx_dict = {
    "from": wallet,
    "priority_fee": priority_fee,
    "max_fee": max_fee,
    "gas_limit": gas_limit,
}

# Publish Data NFT
(data_nft, datatoken, ddo) = ocean.assets.create_url_asset(name, url, tx_dict)

time.sleep(30)

# Set Price to Free
datatoken.create_dispenser(tx_dict)

print("Data NFT published:")
print(f"  data_nft: symbol={data_nft.symbol()}, address={data_nft.address}")
print(f"  datatoken: symbol={datatoken.symbol()}, address={datatoken.address}")
print(f"  did={ddo.did}")
print(f"  Link: https://market.oceanprotocol.com/asset/{ddo.did}")

priority_fee: 30000000000 max_fee: 199099317136 gas_limit: 2290993
Transaction sent: 0xb1f8a31dfa283bae8d15257795b2031f0c6d7478e6fdf18e092ae9ee965ed54a
  Max fee: 199.099317136 gwei   Priority fee: 30.0 gwei   Gas limit: 2290993   Nonce: 1
  ERC721Factory.createNftWithErc20 confirmed   Block: 47139140   Gas used: 1595610 (69.65%)   Gas price: 127.561655414 gwei



INFO:ocean_lib.data_provider.data_encryptor:Asset urls encrypted successfully, encrypted urls str: 0x04f36ad29b15d4df7ea813831c5491f86266a503face3663ab1b6cd450464004918519135f93f879273c44e09adaaf2ea89dbf39046aa47a50a71354e4301fdad3510aba735345762ab9cfe96af3f7e90d01db5eee2cb72feb172a9de2d3f87db2bb6e5ab7cd058ffe9aba352217cfedb83f5a33b30b57a365677be2e2e8b1352b0ec7c955e5a15651fcf9ea3529a0e0375df34a0554d146b7ab6af051406acac64c9477badf08550b647aa3f4865f8a29b75a2747b6c2e36397edf8d781f2ca6de53bd4ab5df8b028a2c87a9c5dcbbc70a7859fc56d3969d4e34eda0242feeca54285bc300b0e36a371b0caa6616aa342103e4a647072140ca075e789e7d66cdc64f2c7cf51f2a02bf17a98ef37506051a5ad65c9afdf71202c146564d7a598f7f53c2349bbf176fdc9fbae95a228c43eca40153f75c0a2ee2ce0a654c172f80d569389d458f8de3548, encryptedEndpoint https://v4.provider.polygon.oceanprotocol.com/api/services/encrypt?chainId=137
INFO:ocean_lib.data_provider.data_encryptor:Asset urls encrypted successfully, encrypted urls str: 0x04a35bcdcdfcebed5c54197f66649efa9c3a5bd

Transaction sent: 0xd57abae033c9aa529c77d42f0b9e1e805d87ebff041b603468b0e5069e653b45
  Max fee: 199.099317136 gwei   Priority fee: 30.0 gwei   Gas limit: 2290993   Nonce: 2
  ERC721Template.setMetaData confirmed   Block: 47139156   Gas used: 211883 (9.25%)   Gas price: 126.062436866 gwei

Transaction sent: 0xe17053fee3d1070acfc281395d1242a7458e32a7ecbdecb03d8212ccd8e57cc0
  Max fee: 199.099317136 gwei   Priority fee: 30.0 gwei   Gas limit: 2290993   Nonce: 3
  ERC20Template.createDispenser confirmed   Block: 47139185   Gas used: 283122 (12.36%)   Gas price: 121.997720348 gwei

Data NFT published:
  data_nft: symbol=Data NFT (Dune), address=0x5E8c219C6f0d445686531449B6991410774A6d84
  datatoken: symbol=DT1, address=0x92CC8c348F851FA29E4f0033256Bf32a2755A923
  did=did:op:78f800cebc1af9dee70e71e39146fccfa350b82cd4ef5c2684305c98a0193eff
  Link: https://market.oceanprotocol.com/asset/did:op:78f800cebc1af9dee70e71e39146fccfa350b82cd4ef5c2684305c98a0193eff
